In [1]:
import tensorflow as tf

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('data.csv')
df.head()

,Gender,Age,Chest Pain,Genereal level of tiredness,"Pulse, resting",Blood Type,Heart disease mom/dad,Smoking,Cholesterol,Alcohol,BMI,Fitness,Use of contact lenses,Diabetes,H�matokritv�rdi,EKG,Go to doctor
0,M,4,Periodic,4,59,0 pos,N,0,1,6,22,121,N,Healthy,38,Anormalities,NO
1,M,6,Often,8,80,0 pos,N,0,6,10,35,118,Y,Type 1,39,Anormalities,OBS
2,F,15,Periodic,0,67,B pos,Y,4,1,0,22,154,N,Healthy,41,Normal,NO
3,M,15,Often,10,69,B pos,Y,6,1,0,22,150,N,Healthy,40,Anormalities,NO
4,F,17,None,1,72,B pos,N,7,1,6,22,124,N,Healthy,42,Anormalities,NO


In [4]:

category_fields = ['Gender', 'Chest Pain', 'Blood Type', 'Heart disease mom/dad', 'Use of contact lenses', 
                   'Diabetes', 'EKG', 'Go to doctor']

for each in category_fields:
    dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
    df = pd.concat([df, dummies], axis=1)
    
df = df.drop(category_fields, axis=1)

pd_y = df[['Go to doctor_YES', 'Go to doctor_OBS', 'Go to doctor_NO']]
pd_x = df.drop(['Go to doctor_YES', 'Go to doctor_OBS', 'Go to doctor_NO'], axis=1)
print(pd_y.head())

   Go to doctor_YES  Go to doctor_OBS  Go to doctor_NO
0                 0                 0                1
1                 0                 1                0
2                 0                 0                1
3                 0                 0                1
4                 0                 0                1


In [5]:
X_train, X_test, y_train, y_test = train_test_split(pd_x, pd_y, 
                                                    test_size=0.2, 
                                                    random_state=0)
X_test, X_eval, y_test, y_eval = train_test_split(X_test, y_test, 
                                                    test_size=0.5, 
                                                    random_state=0)

def chunk(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

train_xb, test_xb, eval_xb = list(chunk(X_train, 16)), list(chunk(X_test, 16)), list(chunk(X_eval, 16))
train_yb, test_yb, eval_yb = list(chunk(y_train, 16)), list(chunk(y_test, 16)), list(chunk(y_eval, 16))

In [6]:
def hidden_layer(inputs, nodes, keep_rate, norm, training):
    outputs = tf.layers.dense(inputs, nodes, kernel_initializer=tf.contrib.layers.xavier_initializer())
    outputs = tf.nn.relu(outputs)
    if norm == 1:
        outputs = tf.layers.batch_normalization(outputs, training=training)
    outputs = tf.layers.dropout(outputs, 1-keep_rate, training=training)
    return outputs

In [45]:
def get_logits(net):
    norm_input = tf.layers.batch_normalization(net.inputs_, training=net.training_)
    hl = hidden_layer(norm_input, net.nodes, net.krate, 0, net.training_)
    hl = hidden_layer(hl, net.nodes, net.krate, 0, net.training_)
    logits = hidden_layer(hl, 3, net.krate, 0, net.training_)
    return logits

In [70]:
def get_cost(logits, targets):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=targets),name='cross_entropy')
    return cross_entropy

In [71]:
def write_cross_entropy(cost):
    with tf.name_scope('cross_entropy'):
        cross_entropy = tf.summary.scalar('cross_entropy', cost)
        return cross_entropy

In [72]:
def set_hypertune_param(cost):
    with tf.name_scope('hypertune'):
        hypertune = tf.summary.scalar('training/hptuning/metric', cost)
        return hypertune

In [73]:
def get_optimizer(cost, lr):
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
        optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(cost)
        return optimizer

In [77]:
class FFDNN:
    def __init__(self):
        print('MODEL INIT')
        tf.reset_default_graph()
        self.epochs = 100
        self.layers = 2
        self.nodes = 300
        self.krate = 0.9
        self.input_shape = [None, train_xb[0].as_matrix().shape[1]]
        self.output_shape = [None, train_yb[0].as_matrix().shape[1]]
        self.inputs_ = tf.placeholder(tf.float32, shape=self.input_shape, name='inputs')
        self.targets_ = tf.placeholder(tf.float32, shape=self.output_shape, name='targets')
        self.train_learning_rate = 0.001
        self.lr_ = tf.placeholder(tf.float32, shape=[], name='lr')
        self.training_ = tf.placeholder(tf.bool, shape=[], name='training')
        self.logits = get_logits(self)
        self.cost = get_cost(self.logits, self.targets_)
        self.cross_entropy = write_cross_entropy(self.cost)
        self.hypertune = set_hypertune_param(self.cost)
        self.optimizer = get_optimizer(self.cost, self.lr_)
        self.saver = tf.train.Saver()

In [78]:
def train(net):
    print('TRAIN')
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        train_writer = tf.summary.FileWriter('train', sess.graph)
        eval_writer = tf.summary.FileWriter('eval', sess.graph)

        for e in range(net.epochs):
            
            print('Epoch ' + str(e))
            for b_idx, b in enumerate(train_xb,0):
                
                x, y = train_xb[b_idx].as_matrix(), train_yb[b_idx].as_matrix()
                train_merged = tf.summary.merge([net.cross_entropy])

                # Train
                feed = {net.inputs_: x, net.targets_: y, net.lr_: net.train_learning_rate, net.training_: True}
                summary_train, _ = sess.run([train_merged, net.optimizer], feed_dict=feed)
                train_writer.add_summary(summary_train, b_idx * (e + 1))

            pred = []
            targets = []
            for b_eval_idx, b_eval in enumerate(eval_xb, 0):
                x, y = eval_xb[b_eval_idx].as_matrix(), eval_yb[b_eval_idx].as_matrix()

                feed = {net.inputs_: x, net.targets_: y, net.lr_: net.train_learning_rate, net.training_: False}
                logits = sess.run(net.logits, feed_dict=feed)
                
                pred.extend(logits[:,0])
                targets.extend(y[:,0])
            
            
            #summary = tf.Summary()
            #summary.value.add(tag='training/hptuning/metric', simple_value=rmse)
            #eval_writer.add_summary(summary, e+1)

        train_writer.close()
        eval_writer.close()
        net.saver.save(sess, './activity6.ckpt')

In [ ]:
net = FFDNN()
train(net)

MODEL INIT
TRAIN
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
